In [1]:
import os
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split

dataset_path = "/kaggle/input/ucmerced/UCMercedDatasetOriginal/Images"

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.RandomAutocontrast(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


full_dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

train_idx, test_idx = train_test_split(
    range(len(full_dataset)), test_size=0.2, random_state=42,
    stratify=[label for _, label in full_dataset.samples]  
)

train_set = Subset(full_dataset, train_idx)
test_set = Subset(full_dataset, test_idx)

print(f"Total samples: {len(full_dataset)}")
print(f"Train set size: {len(train_set)}")
print(f"Test set size: {len(test_set)}")


Total samples: 2100
Train set size: 1680
Test set size: 420


In [2]:
import random
import copy

def label_noise(dataset, noise_level=0.3):
    dataset_noisy = copy.deepcopy(dataset)
    num_samples = len(dataset_noisy.indices) 
    num_noisy = int(noise_level * num_samples)

    indices = random.sample(dataset_noisy.indices, num_noisy)  
    class_count = len(full_dataset.classes)

    for idx in indices:
        _, old_label = full_dataset.samples[idx]  
        new_label = random.randint(0, class_count - 1)
        while new_label == old_label: 
            new_label = random.randint(0, class_count - 1)

        full_dataset.samples[idx] = (full_dataset.samples[idx][0], new_label)  
    return dataset_noisy

train_set_noisy = label_noise(train_set, noise_level=0.3)


In [3]:
batch_size = 32
train_loader = DataLoader(train_set_noisy, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)
print(f"Train set size: {len(train_set_noisy)}")
print(f"Test set size: {len(test_set)}")

Train set size: 1680
Test set size: 420


In [4]:

import timm
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm

models_dict = {
    "EfficientNet-B0": timm.create_model('efficientnet_b0', pretrained=True),
    "DeiT-Tiny": timm.create_model('deit_tiny_patch16_224', pretrained=True),
    "ViT-B/16": timm.create_model('vit_base_patch16_224', pretrained=True)
    
}

num_classes = 21
for name, model in models_dict.items():
    if hasattr(model, "fc"): 
        print("yes 1")
        model.fc = nn.Linear(model.fc.in_features, num_classes)
    # elif hasattr(model, "head_dist"):  
    #     print("yes 2")
    #     model.head = nn.Linear(model.head.in_features, num_classes)
    #     model.head_dist = nn.Linear(model.head_dist.in_features, num_classes)
    elif hasattr(model, "head"):  
        print("yes 3")
        model.head = nn.Linear(model.head.in_features, num_classes)


def train_model(model, train_loader, test_loader, epochs=5, device="cuda"):
    device = torch.device(device if torch.cuda.is_available() else "cpu")
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=0.05)
    scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lambda e: min((e + 1) / 10, 1))
    for epoch in range(epochs):
        model.train()
        correct, total, running_loss = 0, 0, 0.0
        for images, labels in tqdm(train_loader):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            scheduler.step() 
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct = correct + (predicted == labels).sum().item()
            total +=  labels.size(0)
        train_acc = 100*correct / total
        print(f"epoch [{epoch+1}/{epochs}] Loss: {running_loss:.4f} Train Acc: {train_acc:.2f}%")
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            correct = correct + (predicted == labels).sum().item()
            total += labels.size(0)
    return 100 * correct / total


model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/22.9M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

yes 3
yes 3


In [5]:
results = {}
for name, model in models_dict.items():
    print(f"Training {name}...\n") 
   
    if name=="ViT-B/16":
        acc = train_model(model, train_loader, test_loader, epochs=10)
    else:
    #name=="EfficientNet-B0":
        acc = train_model(model, train_loader, test_loader, epochs=5)
    results[name] = acc
    print(f"Test Accuracy: {acc:.2f}%\n")

Training EfficientNet-B0...



100%|██████████| 53/53 [00:20<00:00,  2.58it/s]


epoch [1/5] Loss: 90.2644 Train Acc: 68.93%


100%|██████████| 53/53 [00:12<00:00,  4.34it/s]


epoch [2/5] Loss: 13.7452 Train Acc: 92.68%


...

epoch [8/10] Loss: 48.7252 Train Acc: 68.21%


100%|██████████| 53/53 [00:37<00:00,  1.40it/s]


epoch [9/10] Loss: 45.0727 Train Acc: 71.73%


100%|██████████| 53/53 [00:37<00:00,  1.40it/s]


epoch [10/10] Loss: 43.0712 Train Acc: 72.56%
Test Accuracy: 64.52%

